# Variant/Residue effect class prediction via mutational scanning.

Prediction of protein variants and residue effect on stability and function via evaluation of mutation effects. The program requires as input **thermodynamic stability evaluation** as input ($\Delta \Delta G$ in kcal/mol) generated via e.g. Rosetta or FoldX. It also requires **protein structure** (PDB file) and **conservation scores** ($\Delta \Delta E$) generated using [GEMME](https://academic.oup.com/mbe/article/36/11/2604/5548199?login=true), which can be generated in the official [webserver](http://www.lcqb.upmc.fr/GEMME/submit.html) and converted in the notebook. Output files generated will include input features per variant, prediction per variants and prediction per positions.

# **TO DO:**

- add gemme webserver to gemme local
- import catboost file from github directly

In [1]:
#@title Install dependencies
#@markdown Run the cell to install all the extra necessaries packages, including:
#@markdown - catboost (0.26.1)
#@markdown - MDtraj
#@markdown - Biopython

%%bash -s

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q catboost==0.26.1 ## same version used in the paper
  pip install -q mdtraj
  pip install -q Biopython
fi


In [22]:
#@title Load pipeline functions
#@markdown Run the cell to all the auxiliary pipeline load functions:

import numpy as np
import pandas as pd
import mdtraj as md

## dictionaries
alphabetAA_L_D={'-':0,'_' :0,'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20}
alphabetAA_D_L={v: k for k, v in alphabetAA_L_D.items()}

AA_to_hydrophobicity_scores={'A':44,'C':50,'D':-37,'E':-12,'F':96,'G':0,'H':-16,'I':100,'K':-30,'L':99,'M':74,'N':-35,'P':-46,'Q':-14,'R':-20,'S':-6,'T':13,'V':78,'W':90,'Y':57}

alphabet_class_variant_d_l={0:'WT-like',1:'SBI',2:'Total-loss'}
alphabet_class_residue_d_l={0:'WT-like',1:'Functional site',2:'Total-loss',5:'Not assgined'}


## functions

def remove_WT_score(score,WT_seq):
    for i in range(len(WT_seq)):
        score[i,alphabetAA_L_D[WT_seq[i]]-1]=np.nan
    return score

def load_data(data,wt_seq,start_gap=0,column_score=1):
  df=pd.read_csv(data, delim_whitespace=True, comment='#')
  mutation_load=np.array(df.iloc[:,0])
  score_load=np.array(df.iloc[:,column_score])
  scores=np.empty((len(wt_seq),20),dtype=float)
  scores[:]=np.nan
  for i in range(len(mutation_load)):
      if  mutation_load[i][len(mutation_load[i])-1]!= '=':
          scores[int(mutation_load[i][1:len(mutation_load[i])-1])-1+start_gap, alphabetAA_L_D[mutation_load[i][len(mutation_load[i])-1]]-1]= float(score_load[i])
  return scores

def WCN(pdb_loc,scheme_e,WT):
    r0=7.0
    pdb=md.load(pdb_loc)
    topology=pdb.topology
    chainA=topology.select('chainid 0 and protein')
    pdb_chain0=pdb.atom_slice(chainA)
    pdb_dist,pdb_rp=md.compute_contacts(pdb_chain0,scheme=scheme_e,periodic=False)
    
    cm= md.geometry.squareform(pdb_dist,pdb_rp)[0]
    wcn=np.zeros((len(WT)),dtype=float)
    
    cm_adj=np.empty((len(WT),len(WT)),dtype=float)
    cm_adj[:]=np.nan
    chainA_top=pdb_chain0.topology
    
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if i==0 and j==0:
                print(str(chainA_top.residue(i)))
            cm_adj[int(str(chainA_top.residue(i))[3:])-1,int(str(chainA_top.residue(j))[3:])-1]=cm[i,j]
    for i in range(len(WT)):
        nan_flag=True
        for j in range(len(WT)):
            if np.isnan(cm_adj[i,j])!=True and cm_adj[i,j]!=0.0:
                nan_flag=False
                wcn[i]+=(1-(cm_adj[i,j]*10/r0)**6)/(1-(cm_adj[i,j]*10/r0)**12)
        if nan_flag==True:
            wcn[i]=np.nan
    return wcn

def neighbor_scores(score,ext_range):
  score_neighborhood=np.zeros(len(score),dtype=float)
  for i in range(len(score)):
      if np.isnan(score[i])!=True:
          count_nan=0
          if i==0:
              for j in range(1,ext_range+1):
                  if np.isnan(score[j])==False:
                      score_neighborhood[i]+=score[j]
                  else:
                      count_nan+=1
              if count_nan!=ext_range:    
                  score_neighborhood[i]/=(ext_range)
              else:
                  score_neighborhood[i]=np.nan

          elif i==(len(score)-1):
              for j in range(len(score)-1-ext_range,len(score)-1):
                  if np.isnan(score[j])==False:
                      score_neighborhood[i]+=score[j]
                  else:
                      count_nan+=1
              if count_nan!=ext_range: 
                  score_neighborhood[i]/=ext_range
              else:
                  score_neighborhood[i]=np.nan                
          elif i<ext_range:
              for j in range(0,i+ext_range+1):
                  if j!=i:
                      if np.isnan(score[j])==False:
                          score_neighborhood[i]+=score[j]
                      else:
                          count_nan+=1
              if count_nan!=(i+ext_range):    
                  score_neighborhood[i]/=(i+ext_range)
              else:
                  score_neighborhood[i]=np.nan                        

          elif i>(len(score)-1-ext_range):
              for j in range(i-ext_range,len(score)):
                  if j!=i:
                      if np.isnan(score[j])==False:
                          score_neighborhood[i]+=score[j]
                      else:
                          count_nan+=1
              if count_nan!=(len(score)-i+ext_range):                     
                  score_neighborhood[i]/=(len(score)-i+ext_range)
              else:
                  score_neighborhood[i]=np.nan  
          else:
              for j in range(i-ext_range,i+ext_range+1):
                  if j!=i:
                      if np.isnan(score[j])==False:
                          score_neighborhood[i]+=score[j]
                      else:
                          count_nan+=1
              if count_nan!=(2*ext_range):  
                  score_neighborhood[i]/=(2*ext_range)
              else:
                  score_neighborhood[i]=np.nan             
      else:
          score_neighborhood[i]=np.nan
  return score_neighborhood

def normalize_cutoff(scores,lowcut,highcut):
  normalized_scores=np.copy(scores)
  for i in range(scores.shape[0]):
      for j in range(scores.shape[1]):
          if scores[i,j] < lowcut:
              normalized_scores[i,j]=lowcut
          elif scores[i,j] > highcut:
              normalized_scores[i,j]=highcut
          else:
              normalized_scores[i,j]=scores[i,j]
  return normalized_scores

def position_mean(score):
  score_mean=np.zeros(score.shape[0],dtype=float)
  for i in range(score.shape[0]):
      count=0
      flag_nan=True
      for j in range(score.shape[1]):
          if np.isnan(score[i,j])==False:
              flag_nan=False
              score_mean[i]+=score[i,j]
              count+=1
          else:
              pass
      if flag_nan==True:
          score_mean[i]=np.nan
      score_mean[i]/=count
      
  return score_mean

def features_validation(list_features_x,WT):
    
  X=[]
  mapping_pos=[] 
  
  for i in range(len(WT)):
      for j in range(20):
          if j!=(alphabetAA_L_D[WT[i]]-1):
            
              temp_x=[]
              temp_y=[]
              cond=True

              for elem in list_features_x:
                  if elem.ndim==1:
                      if np.isnan(elem[i])==True:
                          cond=False
                  else:
                      if np.isnan(elem[i,j])==True:
                          cond=False    
          

              if cond==True:
                  
                  for elem in list_features_x:
                      if elem.ndim==1:
                          temp_x.append(elem[i])
                      else:
                          temp_x.append(elem[i,j])
              if len(temp_x)>0:
                  X.append(temp_x)
                  mapping_pos.append([i,j])
      
  return np.array(X),mapping_pos

def retrieve_residue_label_pred(prediction,variant_map,WT,percentage_threshold):
  scores=np.empty((len(WT),20),dtype=float)
  scores[:]=np.nan
  
  i=0
  for cord in variant_map:
      scores[cord[0],cord[1]]=prediction[i]
      i+=1
  
  count_pos=np.zeros((len(WT),4),dtype=float)
  
  pseudomode_value_class=np.empty(len(WT),dtype=float)
  pseudomode_value_class[:]=np.nan
  percentage=np.zeros((len(WT),2),dtype=float)
      
  for i in range(len(pseudomode_value_class)):
      count=0
      for j in range(scores.shape[1]):
          if np.isnan(scores[i,j])!=True:
              count+=1
      if count>0:
          for j in range(scores.shape[1]):
                  if np.isnan(scores[i,j])!=True:
                      count_pos[i,int(scores[i,j])]+=1
          count_pos=count_pos/count
          if np.any(count_pos[i,:]>=0.50):
              pseudomode_value_class[i]=np.argmax(count_pos[i,:])
              percentage[i,0]=count_pos[i,1]
              #print(i, count_pos[i,:],np.max(count_pos[i,:]),np.argmax(count_pos[i,:]),pseudomode_value_class[i])


          else:                
              pseudomode_value_class[i]=5
              percentage[i,0]=count_pos[i,1]
              ## 5 indicates mixed signal with no predominat mutation class
          
          percentage[i,1]=count
    
  return pseudomode_value_class

def count_class_variant_position(variant_matrix,pred_class):
  variant_class_count=[]
  for i in range(variant_matrix.shape[0]):
    variant_class_count.append(np.count_nonzero((variant_matrix[i,:]==pred_class)))
  return np.array(variant_class_count)

In [6]:
#@title Preliminary operations: Load sequence
#@markdown Input the query sequence and a custom jobname if wanted (it will be used also in output file generation)
from google.colab import files
import os.path
import re

query_sequence='MDCLCIVTTKKYRYQDEDTPPLEHSPAHLPNQANSPPVIVNTDTLEAPGYELQVNGTEGEMEYEEITLERGNSGLGFSIAGGTDNPHIGDDPSIFITKIIPGGAAAQDGRLRVNDSILFVNEVDVREVTHSAAVEALKEAGSIVRLYVMRRKPPAEKVMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGRLQIGDKILAVNSVGLEDVMHEDAVAALKNTYDVVYLKVAKPSNAYLSDSYAPPDITTSYSQHLDNEISHSSYLGTDYPTAMTPTSPRRYSPVAKDLLGEEDIPREPRRIVIHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDQILSVNGVDLRNASHEQAAIALKNAGQTVTIIAQYKPEEYSRFEAKIHDLREQLMNSSLGSGTASLRSNPKRGFYIRALFDYDKTKDCGFLSQALSFRFGDVLHVIDASDEEWWQARRVHSDSETDDIGFIPSKRRVERREWSRLKAKDWGSSSGSQGREDSVLSYETVTQMEVHYARPIIILGPTKDRANDDLLSEFPDKFGSCVPHTTRPKREYEIDGRDYHFVSSREKMEKDIQAHKFIEAGQYNSHLYGTSVQSVREVAEQGKHCILDVSANAVRRLQAAHLHPIAIFIRPRSLENVLEINKRITEEQARKAFDRATKLEQEFTECFSAIVEGDSFEEIYHKVKRVIEDLSGPYIWVPARERL'#@param {type:"string"}

jobname='P78'#@param {type:"string"}

input_path = f"{jobname}_inputs"
if not os.path.exists(input_path):
  os.mkdir(input_path)


output_path = f"{jobname}_outputs"
if not os.path.exists(output_path):
  os.mkdir(output_path)


loaded_ddg=False
loaded_pdb=False
loaded_gemme=False
loaded_extra=False


In [7]:
#@title Preliminary operations: load $\Delta \Delta G$ file
#@markdown Run the cell to open the upload request and load $\Delta \Delta G$ mutagenesis file (in kcal/mol) -  file format available on github.\
#@markdown \
#@markdown **File residue numeration should match the input sequence numeration, if not please set up here the amount to add to the rosetta position number:** 
difference_from_query='310'#@param {type:"string"}

print('-> upload ddg file:')
uploaded_ddg = files.upload()

for fn in uploaded_ddg.keys():
    os.rename(fn, f"{jobname}_inputs/{fn}")
    print('--> rosetta ddg score loaded')
    rosetta_scores=load_data(f"{jobname}_inputs/{fn}",query_sequence,start_gap=int(difference_from_query))

rosetta_scores=remove_WT_score(rosetta_scores,query_sequence)
rosetta_scores_norm=normalize_cutoff(rosetta_scores,0,5)
rosetta_scores_mean=position_mean(rosetta_scores_norm)

print('---> rosetta mean evaluated')

loaded_ddg = True


-> upload ddg file:


Saving prism_rosetta_P78352.txt to prism_rosetta_P78352 (1).txt
--> rosetta ddg score loaded
---> rosetta mean evaluated


In [12]:
query_sequence[310:313]

'PRR'

In [13]:
#@title Preliminary operations: load PDB file
#@markdown Run the cell to open the upload request and load protein's PDB file.\
#@markdown \
#@markdown **N.B PDB file residue numeration has to match the input sequence numeration.** 
import mdtraj as md

print('-> upload pdb file:')
pdb_path = f"{jobname}_inputs"
uploaded_pdb = files.upload()

for fn in uploaded_pdb.keys():
  os.rename(fn, f"{jobname}_inputs/{fn}")

  print('--> pdb loaded')
  wcn_scores=WCN(f"{jobname}_inputs/{fn}",'ca',query_sequence)
  print('---> wcn evaluated')
  loaded_pdb = True


-> upload pdb file:


Saving P78352_6qjj_reres.pdb to P78352_6qjj_reres.pdb
--> pdb loaded
ARG309
---> wcn evaluated


In [16]:
#@title Preliminary operations: load GEMME file


print('-> upload GEMME file:')
gemme_path = f"{jobname}_inputs"
uploaded_gemme = files.upload()

#@markdown **1)** Select GEMME file format:
#@markdown - Pipeline format (already formatted, see github examples)
#@markdown - GEMME webserver format (the file will be internally converted)


GEMME_file_format = "Pipeline format" #@param ["Pipeline format", "GEMME webserver format"]

#@markdown **2)** Run the cell to open the upload request and load protein's PDB file.\
#@markdown \
#@markdown **File residue numeration should match the input sequence numeration, if not please set up here the amount to add to the gemme position number:** 
difference_from_query='0'#@param {type:"string"}

for fn in uploaded_gemme.keys():
    os.rename(fn, f"{jobname}_inputs/{fn}")
    print('--> GEMME file correctly loaded')

    
    if GEMME_file_format == "GEMME webserver format":
      pass
    else:
      gemme_scores=load_data(f"{jobname}_inputs/{fn}",query_sequence,start_gap=int(difference_from_query))
  
      print('---> GEMME scores loaded')

gemme_scores=remove_WT_score(gemme_scores,query_sequence)
gemme_scores_mean=position_mean(gemme_scores)

print('---> GEMME mean score evaluated')

loaded_gemme = True

-> upload GEMME file:


Saving prism_gemme_P78352.txt to prism_gemme_P78352.txt
--> GEMME file correctly loaded
---> GEMME scores loaded
---> GEMME mean score evaluated


In [17]:
#@title Evaluating extra necessary features
#@markdown Run the cell to evaluate the extra necessary features, including:
#@markdown - $\Delta \Delta G$ and conserrvation neighbor scores
#@markdown - Weighted contact number

rosetta_neigbour_scores=neighbor_scores(rosetta_scores_mean,1)

gemme_neigbour_scores=neighbor_scores(gemme_scores_mean,1)


hydrophobicity_mut=np.empty((len(query_sequence),20),dtype=float)
hydrophobicity_mut[:]=np.nan

for i in range(len(query_sequence)):
    for j in range(20):
        hydrophobicity_mut[i,j]=AA_to_hydrophobicity_scores[alphabetAA_D_L[j+1]]

loaded_extra=True

In [18]:
loaded_ddg=True

In [23]:
#@title Variant prediction
#@markdown Run the cell to use the trained catboost model and generate predictions for the query protein at variant and residue level.
from catboost import CatBoostClassifier
if ((loaded_ddg) and (loaded_gemme) and (loaded_pdb) and (loaded_extra)):

  X,map=features_validation([gemme_scores,rosetta_scores, gemme_scores_mean,rosetta_scores_mean,hydrophobicity_mut,gemme_neigbour_scores,rosetta_neigbour_scores,wcn_scores],query_sequence)

  cat=CatBoostClassifier()
  cat.load_model("./cat_trained_24jun22.cbm")

  print(f"-> Model loaded")

  prediction=cat.predict(X)

  print(f"--> Prediction succeed ")

  variant_pred=np.empty((len(query_sequence),20),dtype=float)
  variant_pred[:]=np.nan

  for i,(n,m) in enumerate(zip(map,prediction)):
    variant_pred[n[0],n[1]]=m

  print(f"---> Variant score evaluated")

  residue_mode_pred=retrieve_residue_label_pred(prediction,map,query_sequence,0.5)

  print(f"----> Residue score evaluated")

else:
  if loaded_ddg!=True:
    print(f'Error: missing ddGs')
  if loaded_gemme!=True:
    print(f'Error: missing GEMME scores')
  if loaded_pdb!=True:
      print(f'Error: missing pdb and wcn scores')
  if loaded_extra!=True:
      print(f'Error: missing additional feature scores')

-> Model loaded
--> Prediction succeed 
---> Variant score evaluated
----> Residue score evaluated


In [24]:
#@title Download results
#@markdown Download results
mutation_list=[]
residue_list=[]
for i in range(0,len(query_sequence)):
  residue_list.append(query_sequence[i]+str(i+1))
  for j in range(20):
    mutation_list.append(query_sequence[i]+str(i+1)+alphabetAA_D_L[j+1])


print_feature_df=pd.DataFrame({'Mutation': np.array(mutation_list).flatten(),
                        'GEMME variant' : gemme_scores.flatten(),'GEMME mean residue' : np.tile(gemme_scores_mean,(1,20)).flatten(), 'GEMME neighbours' : np.tile(gemme_neigbour_scores,(1,20)).flatten(),
                        'Rosetta ddG variant' : rosetta_scores_norm.flatten() ,'Rosetta ddG mean residue' :  np.tile(rosetta_scores_mean,(1,20)).flatten(),'Rosetta ddG neigbours' : np.tile(rosetta_neigbour_scores,(1,20)).flatten(),
                       'Hydrophobicity variant' : hydrophobicity_mut.flatten(), 'WCN residue' :  np.tile(wcn_scores,(1,20)).flatten()})

print_variant_df=pd.DataFrame({'Mutation': np.array(mutation_list).flatten(),
                        'Variant class ' : np.vectorize(alphabet_class_variant_d_l.get)(variant_pred.flatten()),'Variant class (digit)' : variant_pred.flatten()})

print_residue_df=pd.DataFrame({'Residue': np.array(residue_list).flatten(),
                        'Residue class ' : np.vectorize(alphabet_class_residue_d_l.get)(residue_mode_pred.flatten()),'Residue class (digit)' : residue_mode_pred.flatten(),
                        '# WT-like variant' : count_class_variant_position(variant_pred,0),'# SBI variant' : count_class_variant_position(variant_pred,1),'# Total-loss variant' : count_class_variant_position(variant_pred,2)})

## print files:
#@markdown Select the output format file:
output_format = "csv" #@param ["csv","excel"]
#@markdown Mark the next box if you want a zip file with results (if not file will be store in the colab output folder)
download_results = True #@param {type:"boolean"}
if output_format== 'csv':
  print_feature_df.to_csv(f"{jobname}_outputs/"+f"{jobname}_variant_features.csv",sep=',')
  print_variant_df.to_csv(f"{jobname}_outputs/"+f"{jobname}_variant_predictions.csv",sep=',')
  print_residue_df.to_csv(f"{jobname}_outputs/"+f"{jobname}_residue_predictions.csv",sep=',')

if output_format== 'excel':
  print_feature_df.to_excel(f"{jobname}_outputs/"+f"{jobname}_variant_features.xlsx")
  print_variant_df.to_excel(f"{jobname}_outputs/"+f"{jobname}_variant_predictions.xlsx")
  print_residue_df.to_excel(f"{jobname}_outputs/"+f"{jobname}_residue_predictions.xlsx")
  
  
if download_results:
  os.system( "zip -r {} {}".format( f"{jobname}_outputs.zip" , f"{jobname}_outputs" ) )
  files.download(f"{jobname}_outputs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>